## Notebook para ETL

Importamos librerias necesarias

In [ ]:
# Librería para manejo y análisis de estructuras de datos
import pandas as pd 

# Módulo para leer y escribir un archivo/ Manipular rutas
import os

# Módulo para manipular rutas
from pathlib import Path

### EXTRACCIÓN DE DATOS

In [ ]:
# Función para automatizar extracción de datos en diferentes formatos

def extraccion(path):
    with open (path): #se cierra automáticamente para que no tenga que recordar usar file.close()
        # Para importar archivos csv
        if Path(path).suffix == ".csv": 
            data = pd.read_csv(path)
            print("El archivo: " + path + " de formato csv fue importado correctamente")
        # Para importar archivos json
        elif Path(path).suffix == ".json": 
            data = pd.read_json(path)
            print("El archivo: " + path + " de formato json fue importado correctamente")

    return data

In [ ]:
Path(".").resolve() # Es la ruta donde estoy parado

In [ ]:
os.path.abspath('C:/Users/Usuario/Desktop/Data Engineer/notebooks') # Me normaliza la ruta

In [ ]:
# Establezco variables de ruta
root_dir = Path(".").resolve().parent
root_dir

In [ ]:
# Instanciamos la función extraccion, con el path absoluto de cada datasets

amazon_original = extraccion(r"C:\Users\Usuario\Desktop\Data Engineer\data\raw\amazon_prime_titles.csv")
disney_original  = extraccion(r"C:\Users\Usuario\Desktop\Data Engineer\data\raw\disney_plus_titles.csv")
hulu_original  = extraccion(r"C:\Users\Usuario\Desktop\Data Engineer\data\raw\hulu_titles.csv")
netflix_original  = extraccion(r"C:\Users\Usuario\Desktop\Data Engineer\data\raw\netflix_titles.json")


In [ ]:
# Realizo copia de los dataset para poder conservar los originales

amazon = amazon_original.copy()
disney = disney_original.copy()
hulu = hulu_original.copy() 
netflix = netflix_original.copy()

### TRANSFORMACIÓN DE DATOS

In [ ]:
# Función para normalizar cada dataset

def normalizar(Dataframe,plataform):
    # Reemplazo los valores nulos
    Dataframe.fillna(value = "No data", inplace = True)
    # Elimino duplicados
    Dataframe.drop_duplicates(inplace=True)
    # Creo columna "plataform", sus valores son la plataforma correspondiente
    Dataframe["plataform"] = plataform
    print("El dataset de "+ plataform + " fue normalizado correctamente")

Exploramos la información de cada dataset, corroboro la existencia de datos nulos

In [ ]:
amazon.info()

In [ ]:
disney.info()

In [ ]:
hulu.info()

In [ ]:
netflix.info()

Luego de observar que las columnas de los dataset son datos representativos, analizamos los datos nulos

In [ ]:
# Función para determinar datos nulos 

def nulos(Dataframe,plataforma):
    print("- La cantidad de elementos nulos en "+ plataforma + " es: {}".format(Dataframe.isna().sum().sum()))

In [ ]:
# Instanciamos la función nulos en cada dataset

nulos(amazon,"amazon")
nulos(disney,"disney")
nulos(hulu,"hulu")
nulos(netflix,"netflix")

Instanciamos la función "normalizar" en cada datasets

In [ ]:
normalizar(amazon,"amazon")
normalizar(disney,"disney")
normalizar(hulu,"hulu")
normalizar(netflix,"netflix")

Concatenamos todos los datasets para poder trabajar luego con el dataset resultante

In [ ]:
result = pd.concat([amazon,disney,hulu,netflix])
result

Filtro las columnas necesarias para realizar las consultas solicitadas

In [ ]:
result = result.iloc[:,[1,2,4,7,9,10,12]]
result

### CARGA DE DATASET RESULTANTE EN PROCESS

In [ ]:
final_name_output = "result"
final_output = os.path.join(root_dir,"data","process",final_name_output)
result.to_csv(final_output)

### CONSULTAS

Máxima duración según tipo de film (película/serie), por plataforma y por año: El request debe ser: get_max_duration(año, plataforma, [min o season])

In [ ]:
cat= result["duration"].str.split("",n=1,expand=True)
result["duration"]=cat[0]
result["time_unit"]=cat[1]

In [ ]:
prueba = result.query('plataform == "hulu" and release_year == 2018 ')
prueba[(prueba["duration"].str.contains("min"))]

Cantidad de películas y series (separado) por plataforma El request debe ser: get_count_plataform(plataforma)

In [ ]:
def get_count_plataform(plataforma):
    
    df = result.query("plataforma == @plataforma")
    count = df.type.value_counts(sort=True)

    return plataforma,str(count[0],count[1])

In [ ]:
get_count_plataform("netflix")

Cantidad de veces que se repite un género y plataforma con mayor frecuencia del mismo. El request debe ser: get_listedin('genero')
Como ejemplo de género pueden usar 'comedy', el cuál deberia devolverles un cunt de 2099 para la plataforma de amazon.

In [ ]:
def get_listedin(genero):
    cantidad= result[(result['listed_in'].str.contains(genero))].groupby(['plataform'])["listed_in"].count().head(1)
    cantidad
    return cantidad

In [ ]:
get_listedin("Comedy")

Actor que más se repite según plataforma y año. El request debe ser: get_actor(plataforma, año)

In [ ]:
def get_actor(plataforma,anio):
    return result.query('plataform == @plataforma and release_year == @anio').groupby("cast").count()

In [ ]:
get_actor("netflix",2018)